<img src="../../EnvXGen/images/EnvXGen_logo.png" alt="EnvXGen_logo">

# Crystal Structure Generation Results Analysis

This notebook is designed for post-processing and analysis of crystalline structure generation results.

## Main Capabilities

### Structure Descriptor Computation
* **RDF (Radial Distribution Function)** - radial distribution function
* **ALIGNN** - descriptors based on atomistic linear graph neural network

### Dimensionality Reduction
* **PCA** (Principal Component Analysis)
* **UMAP** (Uniform Manifold Approximation and Projection)

### Analysis and Visualization
* Structure clustering with optimal cluster number determination
* Cosine similarity calculation between generated and original structures
* 2D and 3D visualization of structure projections before and after relaxation
* Energy characteristics analysis

### Optimal Structure Search
* Automatic search for energetically favorable structures in different clusters

The notebook provides a complete pipeline from descriptor computation to creating informative visualizations for analyzing generated crystalline structures.

---

### Downloading modules and functions

In [6]:
import sys
import os

generator_path = os.path.abspath('../../EnvXGen') # change to '../EnvXGen' if you run postprocessing in the results directory
if generator_path not in sys.path:
    sys.path.append(generator_path)

from postprocessing_scripts import *

/home/u7c/yana/EnvXGen/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download database

In [7]:
database_filename = f'Y2H9_relaxation_results_summarized.pkl'

with open(database_filename, 'rb') as database:
    data = pkl.load(database)

### Calculating Descriptors
<b>Here you can choose one of two algorithms — RDF or ALIGNN</b>

As a result, you will get a  `descriptors\` folder containing a subfolder named after the selected algorithm (`RDF\` or `ALIGNN\`), which will contain the following files:

`generated_structures.csv` - descriptors of generated structures  
`POSCAR_init.csv` - descriptors of initial structure  
`relaxed_structures.csv` - descriptors of structures after relaxation  
`relaxed_structures_init_atoms.csv` - descriptors of initial atoms in relaxed structures  
`relaxed_structures_similarities.csv` - similarities of initial atoms in relaxed structures vs POSCAR_init  

In [29]:
calculate_descriptors(descriptor_algorithm='ALIGNN',
                      poscar_init_path='POSCAR_init',  # change to '../POSCAR_init' if you run postprocessing in the results directory
                      database_filename='Y2H9_relaxation_results_summarized.pkl',
                      device='cpu',
                      batch_size=32,
                      n_jobs=-1)

Using device: cpu
Using 12 parallel processes
Loading ALIGNN model...
Calculating descriptor for POSCAR_init...
Processing generated structures...
Calculating descriptors for generated_structures.csv...


Processing generated_structures.csv: 100%|██████████| 13/13 [00:22<00:00,  1.73s/it, batch=13/13]


ALIGNN descriptors for generated_structures.csv saved successfully

Processing relaxed structures...
Calculating descriptors for relaxed_structures.csv...


Processing relaxed_structures.csv: 100%|██████████| 13/13 [00:24<00:00,  1.87s/it, batch=13/13]


ALIGNN descriptors for relaxed_structures.csv saved successfully

Processing initial atoms in relaxed structures and calculating similarities...


Processing relaxed structures with init indices: 100%|██████████| 13/13 [00:11<00:00,  1.16it/s]

ALIGNN descriptors for initial atoms in relaxed structures saved successfully
Descriptor calculation completed!


### Reducing Descriptors
<b>Here you can choose one of two algorithms — PCA or UMAP</b>

As a result, you will get new files in `descriptors\RDF\` or `descriptors\ALIGNN\` folder:

`generated_structures_PCA.csv` or `generated_structures_UMAP.csv`  
`relaxed_structures_PCA.csv` or `relaxed_structures_UMAP.csv` 

This files will contain all neccessary information for 2D or 3D visualization.  
You may already have precomputed RDF or ALIGNN descriptors, so be sure to specify which type of descriptors you want to reduce.

In [34]:
reducing_descriptors(descriptor_algorithm='ALIGNN',
                     reducer_algorithm='UMAP',
                     poscar_init_path='POSCAR_init', # default path
                    )

Reducing data


<h3>Visualization of Crystal Structures by Descriptors</h3>

<b>This function allows you to build 2D or 3D plots of crystal structures based on their descriptors</b>
<br>

Users can customize the visualization using the <b>following parameters:</b>


<li><b>descriptor</b>: Type of descriptors used to build the plot. Possible values:
  <ul>
    <code>"RDF"</code> – Radial Distribution Function;<br>
    <code>"ALIGNN"</code> – Descriptors obtained from the ALIGNN model.
  </ul>
</li><br>

  <li><b>reducer</b>: Dimensionality reduction algorithm:
    <ul>
      <code>"PCA"</code> – Principal Component Analysis;<br>
      <code>"UMAP"</code> – Uniform Manifold Approximation and Projection.
    </ul>
  </li><br>

  <li><b>dim</b> (int): Target dimension of the projection:
    <ul>
      <code>2</code> – Two-dimensional projection;<br>
      <code>3</code> – Three-dimensional projection.
    </ul><br>
  </li>

  <li><b>enthalpy</b> (bool): Whether to add formation enthalpy of structures as an additional axis.
    <ul>
      If <code>dim = 2</code> and <code>enthalpy = True</code>, the plot will be 3D: X, Y, Enthalpy;<br>
      If <code>dim = 3</code>, adding an additional axis is not possible – the <code>enthalpy</code> parameter is ignored.
    </ul><br>
  </li>

  <li><b>structures_before_relaxation</b> and <b>structures_after_relaxation</b> (bool):
    <ul>
      Specify whether to display structures before and/or after relaxation.</br>
      At least one of these parameters must be set to <code>True</code>. A plot with neither relaxed nor unrelaxed structures is not allowed.</li>
    </ul>
  </li>



In [40]:
df_results = plot_results(database_filename=database_filename,
                          descriptor_algorithm='ALIGNN',
                          reducer_algorithm='UMAP',
                          descriptors_dimensionality=3,
                          include_enthalpy=False,
                          structures_before_relaxation=False,
                          structures_after_relaxation=True,
                          poscar_init_path='POSCAR_init' # default path
                          )

You can also analyze the <code>df_results</code> dataframe, which contains information in the following format:<p>
<code>ID</code> – ID of the structure  
<code>x</code>, <code>y</code> or <code>x</code>, <code>y</code>, <code>z</code> – coordinates, calculated using reducer  
<code>Energy</code> and <code>Volume</code> of the structure  
<code>SG</code> – space group of the structure
<code>SG_symbol</code> – international symbol of space group  
<code>Cosine_similarity</code> – similarity between initial atoms in relaxed structures vs POSCAR_init  
(насколько сильно итоговая структура похожа на исходную структуру)

### Get top structures from clusters

In [41]:
df_top_structures = find_different_optimal_structures(
    database_filename=database_filename,
    descriptor_algorithm='ALIGNN',
    k=5
)

In [42]:
df_top_structures

,ID,cluster,epoch,CalcFold,generated_structure_energy,generated_structure_volume,generated_structure_SG,generated_structure_symbol,generated_structure,relaxed_structure_energy,relaxed_structure_volume,relaxed_structure_SG,relaxed_structure_symbol,relaxed_structure,warnings
0,ID-179,0,0,179,13.155815,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.555091,32.631473,1,P1,"(Atom('Y', [0.23142694919822357, 1.54121451425...",NaN
1,ID-140,0,0,140,15.446305,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.555058,32.629457,1,P1,"(Atom('Y', [1.3555357373933676, 1.864854031097...",NaN
2,ID-280,0,0,280,15.960086,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.554993,32.630501,1,P1,"(Atom('Y', [0.6187955155121421, 1.297978266594...",NaN
3,ID-119,0,0,119,11.899413,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.551962,32.668561,1,P1,"(Atom('Y', [1.194065794093604, 2.9020206531840...",NaN
4,ID-165,0,0,165,11.201674,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.551813,32.663640,1,P1,"(Atom('Y', [0.5746346671770619, 1.931561303470...",NaN
5,ID-182,1,0,182,11.417099,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.380362,32.696481,8,Cm,"(Atom('Y', [0.5166753351814241, 2.088369927574...",NaN
6,ID-258,1,0,258,15.646954,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.380250,32.697826,8,Cm,"(Atom('Y', [-0.06468700907774813, 2.1894589681...",NaN
7,ID-286,1,0,286,9.905476,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.380160,32.699185,8,Cm,"(Atom('Y', [0.527457674389845, 2.2392039488190...",NaN
8,ID-245,1,0,245,14.423962,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.380077,32.701238,8,Cm,"(Atom('Y', [0.06223232547751598, 2.16911083218...",NaN
9,ID-125,1,0,125,16.357518,32.168721,1,P1,"(Atom('Y', [1.0541109894588898, 1.826157809130...",-4.380049,32.701237,8,Cm,"(Atom('Y', [0.1686616558458855, 2.414414908494...",NaN
